In [1]:
pip install kaggle


In [5]:
import os
os.environ["KAGGLE_API_TOKEN"] = "KGAT_c2c31bd8e1a922fb3182630f69e9ffd9"


In [6]:
!kaggle competitions list


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [8]:
import json, os

kaggle_json = {
    "username": "sanikabharne",
    "key": "KGAT_c2c31bd8e1a922fb3182630f69e9ffd9"
}

os.makedirs("/root/.config/kaggle", exist_ok=True)

with open("/root/.config/kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_json, f)

os.chmod("/root/.config/kaggle/kaggle.json", 0o600)


In [9]:
!kaggle competitions list

ref                                                                                 deadline             category                reward  teamCount  userHasEntered  
----------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3       2026-04-15 23:59:00  Featured         2,207,152 Usd       1898           False  
https://www.kaggle.com/competitions/vesuvius-challenge-surface-detection            2026-02-13 23:59:00  Research           200,000 Usd       1276           False  
https://www.kaggle.com/competitions/stanford-rna-3d-folding-2                       2026-03-25 23:59:00  Featured           100,000 Usd        781           False  
https://www.kaggle.com/competitions/med-gemma-impact-challenge                      2026-02-24 23:59:00  Featured           100,000 Usd        108           False  
https://ww

In [10]:
!kaggle competitions download -c binary-classification-with-a-bank-dataset-clone
!unzip binary-classification-with-a-bank-dataset-clone.zip


  0% 0.00/14.7M [00:00<?, ?B/s]
100% 14.7M/14.7M [00:00<00:00, 226MB/s]
Archive:  binary-classification-with-a-bank-dataset-clone.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [11]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

train.head()


Train shape: (750000, 18)
Test shape: (250000, 17)


,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [12]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


In [13]:
target = "target"   # confirm this exists
features = [c for c in train.columns if c != target]


In [14]:
cat_cols = train[features].select_dtypes(include="object").columns
num_cols = train[features].select_dtypes(exclude="object").columns

print("Categorical:", len(cat_cols))
print("Numerical:", len(num_cols))


Categorical: 9
Numerical: 9


In [15]:
from sklearn.preprocessing import LabelEncoder

for col in cat_cols:
    le = LabelEncoder()
    combined = pd.concat([train[col], test[col]], axis=0).astype(str)
    le.fit(combined)

    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))


In [17]:
train.columns


Index(['id', 'age', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'y'],
      dtype='object')

In [18]:
target = "y"
features = [c for c in train.columns if c != target]


In [19]:
train[target].value_counts()


,count
y,
0,659512
1,90488


In [20]:
from sklearn.model_selection import StratifiedKFold

X = train[features]
y = train[target]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np

oof_preds = np.zeros(len(train))

for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    model = LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    )

    model.fit(X_tr, y_tr)
    val_pred = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_pred

    auc = roc_auc_score(y_val, val_pred)
    print(f"Fold {fold+1} ROC-AUC: {auc:.4f}")

print("Overall ROC-AUC:", roc_auc_score(y, oof_preds))


Fold 1 ROC-AUC: 0.8940
Fold 2 ROC-AUC: 0.8924
Fold 3 ROC-AUC: 0.8970
Fold 4 ROC-AUC: 0.8932
Fold 5 ROC-AUC: 0.8914
Overall ROC-AUC: 0.8934581883004904


In [22]:
final_model = LogisticRegression(max_iter=1000, n_jobs=-1)
final_model.fit(X, y)

test_preds = final_model.predict_proba(test[features])[:, 1]


In [23]:
submission = pd.read_csv("sample_submission.csv")
submission[target] = test_preds

submission.to_csv("submission.csv", index=False)
submission.head()


,id,y
0,750000,0.037657
1,750001,0.351514
2,750002,0.027397
3,750003,0.008065
4,750004,0.072713
